In [20]:
from praw import Reddit
from praw.models import Subreddit, Submission
import os
from dotenv import load_dotenv
from datetime import datetime, date, timedelta
from dotenv import dotenv_values
import datetime as dt
import pandas as pd
from datetime import datetime, date, timedelta
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import re

In [21]:
# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define stopwords and stemmer for English
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

# Preprocessing function for each comment
def normalize_document(doc):
    # Remove non-alphabetic characters (keep only letters)
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, flags=re.I | re.A)
    doc = doc.lower().strip()
    
    # Tokenize the document
    tokens = word_tokenize(doc)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    # Apply stemming
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Rebuild the document from the processed tokens
    doc = ' '.join(tokens)
    return doc

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chaimaa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chaimaa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Applying NLP on comments column

In [ ]:
full_df =  pd.read_csv("full_dataset_updated.csv")
full_df = full_df.dropna(subset=['comment_body'])
full_df['post_created'] = pd.to_datetime(full_df['post_created'], format='%Y-%m-%d %H:%M:%S')
full_df = full_df[full_df["comment_body"]!="[removed]"]
full_df = full_df[full_df["comment_body"]!="[deleted]"]
full_df['processed_comments'] = full_df['comment_body'].apply(normalize_document)
full_df[['comment_body', 'processed_comments']].head()


In [ ]:
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]
company_dfs = {company: full_df[full_df['company'] == company] for company in full_df['company'].unique()}
for company_name in keywords:
    company_df = company_dfs.get(company_name)
    
    globals()[company_name] = company_df
    


In [ ]:
globals()['microsoft'].head()

,post_title,post_score,post_id,post_num_comments,post_created,post_body,comment_id,comment_body,comment_score,company,processed_comments
168,Google in Crisis,718,1b4d5mn,533,2024-03-02 02:30:14,https://www.bigtechnology.com/p/inside-the-cri...,kt1g4fi,Clearly... \n\nWaymo actually has a fully auto...,4,microsoft,clear waymo actual fulli autonom ride share se...
169,Google in Crisis,718,1b4d5mn,533,2024-03-02 02:30:14,https://www.bigtechnology.com/p/inside-the-cri...,kszx1da,ChatGPT 4 can search the web with the paid ver...,18,microsoft,chatgpt search web paid version
170,Google in Crisis,718,1b4d5mn,533,2024-03-02 02:30:14,https://www.bigtechnology.com/p/inside-the-cri...,kt06obu,>Just remember all of ChatGPT’s data is over 2...,4,microsoft,rememb chatgpt data year old actual true reall...
171,Google in Crisis,718,1b4d5mn,533,2024-03-02 02:30:14,https://www.bigtechnology.com/p/inside-the-cri...,kszy5m0,"No it isn’t, GPT4’s latest training data is f...",17,microsoft,isnt gpts latest train data april brows web li...
172,Google in Crisis,718,1b4d5mn,533,2024-03-02 02:30:14,https://www.bigtechnology.com/p/inside-the-cri...,kt06d1j,">I've used Chat GPT sometimes, but damn it's b...",3,microsoft,ive use chat gpt sometim damn bad free paid ve...


### Using VADER (Better for Social Media)

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def analyze_sentiment_vader(text):
    scores = sia.polarity_scores(text)
    return scores['compound']  


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Chaimaa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
for company_name in keywords: 
    if company_name in globals():
        df = globals()[company_name]
        if df is None:
            print(f"{company_name} DataFrame is None.")
            continue
        df['sentiment_score'] = df['processed_comments'].apply(analyze_sentiment_vader)

        # Classify sentiment
        df['sentiment'] = df['sentiment_score'].apply(lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'neutral'))

        #df[['processed_comments', 'sentiment_score', 'sentiment']].head()
        globals()[company_name] = df

C:\Users\Chaimaa\AppData\Local\Temp\ipykernel_8980\3009728685.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_score'] = df['processed_comments'].apply(analyze_sentiment_vader)
C:\Users\Chaimaa\AppData\Local\Temp\ipykernel_8980\3009728685.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment_score'].apply(lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'neutral'))
C:\Users\Chaimaa\AppData\Local\Temp\ipykernel_8980\3009728685.py:7: SettingWithCopyW

In [ ]:
globals()['meta'][['processed_comments', 'sentiment_score', 'sentiment']].head()

KeyError: "['sentiment_score', 'sentiment'] not in index"

###  Using Hugging Face Transformers (Advanced Model)

In [ ]:
!pip install transformers torch

In [ ]:
!pip install emoji==0.6.0


     ---------------------------------------- 0.0/51.0 kB ? eta -:--:--
     ------------------------ --------------- 30.7/51.0 kB ? eta -:--:--
     -------------------------------------- 51.0/51.0 kB 522.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49745 sha256=440326bc95df84cb28536145d8d485c747974d3d5dec46f7b293d9782899e464
  Stored in directory: c:\users\chaimaa\appdata\local\pip\cache\wheels\1b\bd\d9\310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji



[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from transformers import pipeline
sentiment_analyzer = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

for company in keywords: 
    if company in globals():
        df = globals()[company]
        if df is None:
            print(f"{company} DataFrame is None.")
            continue
        df['sentiment_results'] = df['processed_comments'].apply(lambda x: sentiment_analyzer(x[:128]))

        df['sentiment'] = df['sentiment_results'].apply(lambda x: x[0]['label'])
        df['confidence'] = df['sentiment_results'].apply(lambda x: x[0]['score'])

        globals()[company] = df



KeyboardInterrupt: 

In [ ]:
globals()['meta'][['processed_comments', 'sentiment', 'confidence']].head()


KeyError: "['confidence'] not in index"